In [1]:
import itertools
import more_itertools
import math as m
import os 
import networkx as nx

In [2]:
def read_input(graphfile,number_subpath):
    
    trip_data = open(graphfile,'r').read().split('\n')
    i = 0
    listOfGraphs = {}
    k = 0
    
    while(True):
        if "#" in trip_data[i]:
            i = i+1
            N = int(trip_data[i])
            edges = list()
            subpaths = {}
            while(True):
                i = i+1;
                if "#" in trip_data[i]:
                    break;
                if "" == trip_data[i]:
                    break;
                if "subpaths" in trip_data[i]:
                    for j in range(0,number_subpath):
                        i = i + 1
                        line = trip_data[i].split(" ")
                        subpaths[j] = (line[0:len(line)-1])
                    i = i + 4
                if i >= len(trip_data)-1:
                    break;
                line = trip_data[i].split(" ")
                edges.append((line[0],line[1],line[2]))
            G = {'Nodes':N,'list of edges': edges,'subpaths':subpaths}
            print(subpaths)
            listOfGraphs[k] = G
            k +=1 
        if i >= len(trip_data)-1:
                    break; 

    return listOfGraphs;

In [3]:
def flowMultipleDecomposition(data,K):
    
    
    #libraries
    import gurobipy as gp
    from gurobipy import GRB
    
    # calculate the minimal flow decomposition based on such graph
    V = data['vertices']
    E = data['edges']
    f = data['flows']
    subpaths = data['subpaths']    
    W = 1e5
    S = {str(min(map(int,V)))}
    D = {str(max(map(int,V)))}
    
    
    try:
        #create extra sets
        T = [(i,j,k) for (i,j) in E for k in range(0,K)]
        SC = [k for k in range(0,K)]
        R = [(k,s) for k in range(0,K) for s in range(0,len(subpaths))]
        
        # Create a new model
        model = gp.Model("MFD")
        model.Params.LogToConsole = 0

        # Create variables
        x = model.addVars(T,vtype=GRB.BINARY, name="x")
        w = model.addVars(SC,vtype=GRB.INTEGER, name="w",lb=0)
        z = model.addVars(T,vtype=GRB.CONTINUOUS, name="z",lb=0)
        r = model.addVars(R,vtype=GRB.BINARY,name="r")
    
        model.setObjective(GRB.MINIMIZE)
       
        # flow conservation
        model.addConstrs(gp.quicksum(x[i,j,k] for j in V if i!=j and(i,j) in E) - gp.quicksum(x[j,i,k] for j in V if i!=j and (j,i) in E) == 1 for k in range(0,K) for i in V if i in S)  
        model.addConstrs(gp.quicksum(x[i,j,k] for j in V if i!=j and (i,j) in E) - gp.quicksum(x[j,i,k] for j in V if i!=j and (j,i) in E) == -1 for k in range(0,K) for i in V if i in D)
        model.addConstrs(gp.quicksum(x[i,j,k] for j in V if i!=j and (i,j) in E) - gp.quicksum(x[j,i,k] for j in V if i!=j and (j,i) in E) == 0 for k in range(0,K) for i in V if i not in D and i not in S)

        # flow balance
        model.addConstrs(f[i,j] == gp.quicksum(z[i,j,k] for k in range(0,K)) for (i,j) in E)

        # linearization
        model.addConstrs(z[i,j,k] <= W*x[i,j,k] for(i,j) in E for k in range(0,K))
        model.addConstrs(w[k] - (1 - x[i,j,k])*W <= z[i,j,k] for(i,j) in E for k in range(0,K))
        model.addConstrs(z[i,j,k] <= w[k] for(i,j) in E for k in range(0,K))
        model.addConstrs(z[i,j,k] >= 0 for(i,j) in E for k in range(0,K))
        
        # subpath constraints
        for k in range(0,K):
            for sp_len in range(0,len(subpaths)):
                subpath_edges = list(more_itertools.pairwise(subpaths[sp_len]))
                model.addConstr(gp.quicksum(x[i,j,k] for (i,j) in subpath_edges) >= len(subpath_edges)*r[k,sp_len])
        
        model.addConstrs(gp.quicksum(r[k,sp_len] for k in range(0,K)) >=1 for sp_len in range(0,len(subpaths)))
            
        
        # objective function
        model.optimize()
        
        w_sol = [0]*len(range(0,K))
        x_sol = {}
        paths = [list() for i in range(0,K)]
    
        
        if model.status == GRB.OPTIMAL:
            data['message'] = 'solved'
            data['runtime'] = model.Runtime;

            for v in model.getVars():
                if 'w' in v.VarName:
                    for k in range(0,K):
                        if str(k) in v.VarName:
                            w_sol[k] = v.x
                
                if 'x' in v.VarName:          
                    for (i,j,k) in T:
                        if str(i)+","+str(j)+","+str(k) in v.VarName:
                            x_sol[i,j,k] = v.x
                
            for(i,j,k) in T:
                if x_sol[i,j,k] == 1:
                    paths[k].append((i,j))
            
            data['weights'] = w_sol
            data['solution'] = paths
            
        if model.status == GRB.INFEASIBLE:
            data['message'] = 'unsolved'
        
    except gp.GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))

    except AttributeError:
        print('Encountered an attribute error')
    
    return data;
    

In [4]:
def FD_Algorithm(data):
    
    listOfEdges = data['edges']
    solutionMap = data['graph']
    solutionSet = 0
    Kmin = data['minK']
    solutionWeights = 0

    for i in range(1,len(listOfEdges)+1):
        data = flowMultipleDecomposition(data,i)
        if data['message'] == "solved":
            solutionSet = data['solution']
            solutionWeights = data['weights']
            break;
        
    
    return data

In [5]:
def SolveInstances(Graphs,outfile,recfile):
    
    fp = open(outfile,'w+')
    fc = open(recfile,'w+')
    
    for i in range(0,1): 
        
        f = {}
        Edges = set()
        V = set()
        listOfEdges = Graphs[i]['list of edges']
        for k in range(0,len(listOfEdges)):
            (a,b,c) = (listOfEdges[k])
            Edges.add((a,b))
            V.add(a)
            V.add(b)
            f[a,b] = int(float(c))
            
        
        # creation of graphs
        G = nx.DiGraph()
        G.add_edges_from(Edges,weights = f)
        G.add_nodes_from(V)
        
        
        # calculating topological order
        G_top = nx.topological_sort(G)
        G_top_list = list(nx.topological_sort(G))
        topological_cuts = sorted(G.degree, key=lambda x: x[1], reverse=True)[0][1]
        # definition of data
        
        data = {'edges' : Edges,
                'flows' : f,
                'vertices' : V,
                'graph' : G,
                'subpaths': Graphs[i]['subpaths'],
                'Kmax' : len(Edges),
                'weights' : {},
                'message': {},
                'minK': topological_cuts,
                'runtime': 0,
        }
        
        data = FD_Algorithm(data)        
        
        w_sol = data['weights']
        paths = data['solution']
        print(len(data['weights']),data['runtime'],file=fp)
        for k in range(0,len(data['weights'])):
            print(w_sol[k],paths[k],file=fc)
        print("------",file=fc)
        
    
    return 0

In [6]:
path = "../Example/Subpath/"
text_files = [f for f in os.listdir(path) if f.endswith('.graph')]
outputfile = "results_gurobi_subpath.txt"
numberOfSubpaths = 4
recordfile = "results_gurobi_subpath_details.txt"

for i in range(0,len(text_files)):    
    filename = text_files[i]
    data = read_input(''.join([path,filename]),numberOfSubpaths)
    data = SolveInstances(data,outputfile,recordfile)
    
print("Done")

{0: ['1', '3'], 1: ['1', '2'], 2: ['0', '1'], 3: ['2', '3']}
Academic license - for non-commercial use only - expires 2022-10-10
Using license file /Users/cunhfern/gurobi.lic
Done
